Before we get started, a couple of reminders to keep in mind when using iPython notebooks:

- Remember that you can see from the left side of a code cell when it was last run if there is a number within the brackets.
- When you start a new notebook session, make sure you run all of the cells up to the point where you last left off. Even if the output is still visible from when you ran the cells in your previous session, the kernel starts in a fresh state so you'll need to reload the data, etc. on a new session.
- The previous point is useful to keep in mind if your answers do not match what is expected in the lesson's quizzes. Try reloading the data and run all of the processing steps one by one in order to make sure that you are working with the same variables and data that are at each quiz stage.


## Load Data from CSVs

In [11]:
import unicodecsv

## Longer version of code (replaced with shorter, equivalent version below)

# enrollments = []
# f = open('enrollments.csv', 'rb')
# reader = unicodecsv.DictReader(f)
# for row in reader:
#     enrollments.append(row)
# f.close()

with open('enrollments.csv', 'rb') as f:
    reader = unicodecsv.DictReader(f)
    enrollments = list(reader)

In [12]:
#####################################
#                 1                 #
#####################################

## Read in the data from daily_engagement.csv and project_submissions.csv 
## and store the results in the below variables.
## Then look at the first row of each table.

with open('daily_engagement.csv', 'rb') as f:
    reader = unicodecsv.DictReader(f)
    daily_engagement = list(reader)
with open('project_submissions.csv', 'rb') as f:
    reader = unicodecsv.DictReader(f)
    project_submissions = list(reader)
print(daily_engagement[0])
print(project_submissions[0])

## Alternative
def read_csv(filename):
    with open(filename, 'rb') as f:
        reader = unicodecsv.DictReader(f)
        return list(reader)
daily_engagement = read_csv('daily_engagement.csv')
project_submissions = read_csv('project_submissions.csv')

{u'lessons_completed': u'0.0', u'num_courses_visited': u'1.0', u'total_minutes_visited': u'11.6793745', u'projects_completed': u'0.0', u'acct': u'0', u'utc_date': u'2015-01-09'}
{u'lesson_key': u'3176718735', u'processing_state': u'EVALUATED', u'account_key': u'256', u'assigned_rating': u'UNGRADED', u'completion_date': u'2015-01-16', u'creation_date': u'2015-01-14'}


## Fixing Data Types

In [13]:
from datetime import datetime as dt

# Takes a date as a string, and returns a Python datetime object. 
# If there is no date given, returns None
def parse_date(date):
    if date == '':
        return None
    else:
        return dt.strptime(date, '%Y-%m-%d')
    
# Takes a string which is either an empty string or represents an integer,
# and returns an int or None.
def parse_maybe_int(i):
    if i == '':
        return None
    else:
        return int(i)

# Clean up the data types in the enrollments table
for enrollment in enrollments:
    enrollment['cancel_date'] = parse_date(enrollment['cancel_date'])
    enrollment['days_to_cancel'] = parse_maybe_int(enrollment['days_to_cancel'])
    enrollment['is_canceled'] = enrollment['is_canceled'] == 'True'
    enrollment['is_udacity'] = enrollment['is_udacity'] == 'True'
    enrollment['join_date'] = parse_date(enrollment['join_date'])
    
enrollments[0]

{u'account_key': u'448',
 u'cancel_date': datetime.datetime(2015, 1, 14, 0, 0),
 u'days_to_cancel': 65,
 u'is_canceled': True,
 u'is_udacity': True,
 u'join_date': datetime.datetime(2014, 11, 10, 0, 0),
 u'status': u'canceled'}

In [14]:
# Clean up the data types in the engagement table
for engagement_record in daily_engagement:
    engagement_record['lessons_completed'] = int(float(engagement_record['lessons_completed']))
    engagement_record['num_courses_visited'] = int(float(engagement_record['num_courses_visited']))
    engagement_record['projects_completed'] = int(float(engagement_record['projects_completed']))
    engagement_record['total_minutes_visited'] = float(engagement_record['total_minutes_visited'])
    engagement_record['utc_date'] = parse_date(engagement_record['utc_date'])
    
daily_engagement[0]

{u'acct': u'0',
 u'lessons_completed': 0,
 u'num_courses_visited': 1,
 u'projects_completed': 0,
 u'total_minutes_visited': 11.6793745,
 u'utc_date': datetime.datetime(2015, 1, 9, 0, 0)}

In [15]:
# Clean up the data types in the submissions table
for submission in project_submissions:
    submission['completion_date'] = parse_date(submission['completion_date'])
    submission['creation_date'] = parse_date(submission['creation_date'])

project_submissions[0]

{u'account_key': u'256',
 u'assigned_rating': u'UNGRADED',
 u'completion_date': datetime.datetime(2015, 1, 16, 0, 0),
 u'creation_date': datetime.datetime(2015, 1, 14, 0, 0),
 u'lesson_key': u'3176718735',
 u'processing_state': u'EVALUATED'}

Note when running the above cells that we are actively changing the contents of our data variables. If you try to run these cells multiple times in the same session, an error will occur.

## Investigating the Data

In [34]:
#####################################
#                 3                 #
#####################################

## Rename the "acct" column in the daily_engagement table to "account_key".
for engagement in daily_engagement:
    engagement['account_key'] = engagement['acct']
    del engagement['acct']
print(daily_engagement[0]['account_key'])

0


In [112]:
#####################################
#                 2                 #
#####################################

## Find the total number of rows and the number of unique students (account keys)
## in each table.
def unique_values(table, value):
    unique_values = set()
    for row in table:
        unique_values.add(row[value])
    return unique_values

enrollment_num_rows = len(enrollments)
enrollment_num_unique_students = len(unique_values(enrollments, 'account_key'))
print(enrollment_num_rows)
print(enrollment_num_unique_students)

engagement_num_rows = len(daily_engagement)
engagement_num_unique_students = len(unique_values(daily_engagement, 'account_key'))
print(engagement_num_rows)
print(engagement_num_unique_students)

submission_num_rows = len(project_submissions)
submission_num_unique_students = len(unique_values(project_submissions, 'account_key'))
print(submission_num_rows)
print(submission_num_unique_students)

1640
1302
136240
1237
3642
743


## Problems in the Data

## Missing Engagement Records

In [52]:
#####################################
#                 4                 #
#####################################

## Find any one student enrollments where the student is missing from the daily engagement table.
## Output that enrollment.
i = 0
missings = set()
while i < enrollment_num_rows:
    if enrollments[i]['account_key'] not in unique_values(daily_engagement, 'account_key'):
       #missings.add(enrollments[i]['account_key'])
        missings.add(i)
    i += 1
print(missings)

set([1281, 1410, 918, 193, 1032, 1036, 654, 941, 1424, 278, 1177, 921, 283, 540, 1498, 1440, 1075, 291, 1543, 680, 681, 426, 903, 940, 557, 1454, 1455, 1096, 1331, 1011, 565, 1213, 65, 1218, 1244, 968, 969, 1608, 75, 1356, 1613, 1358, 847, 1105, 1106, 547, 468, 1486, 470, 1367, 216, 217, 218, 475, 220, 1501, 1563, 1378, 1459, 1254, 364, 1389, 623, 1009, 467, 884, 376, 1145, 1147, 380, 639])


In [69]:
#### print(list(missings)[:5])
for i in range(len(missings)):
    print(enrollments[i])

{u'status': u'canceled', u'is_udacity': True, u'is_canceled': True, u'join_date': datetime.datetime(2014, 11, 10, 0, 0), u'account_key': u'448', u'cancel_date': datetime.datetime(2015, 1, 14, 0, 0), u'days_to_cancel': 65}
{u'status': u'canceled', u'is_udacity': True, u'is_canceled': True, u'join_date': datetime.datetime(2014, 11, 5, 0, 0), u'account_key': u'448', u'cancel_date': datetime.datetime(2014, 11, 10, 0, 0), u'days_to_cancel': 5}
{u'status': u'canceled', u'is_udacity': True, u'is_canceled': True, u'join_date': datetime.datetime(2015, 1, 27, 0, 0), u'account_key': u'448', u'cancel_date': datetime.datetime(2015, 1, 27, 0, 0), u'days_to_cancel': 0}
{u'status': u'canceled', u'is_udacity': True, u'is_canceled': True, u'join_date': datetime.datetime(2014, 11, 10, 0, 0), u'account_key': u'448', u'cancel_date': datetime.datetime(2014, 11, 10, 0, 0), u'days_to_cancel': 0}
{u'status': u'current', u'is_udacity': True, u'is_canceled': False, u'join_date': datetime.datetime(2015, 3, 10, 0,

## Checking for More Problem Records

In [67]:
#####################################
#                 5                 #
#####################################

## Find the number of surprising data points (enrollments missing from
## the engagement table) that remain, if any.
missing_but_remains = set()
for i in list(missings):
    if enrollments[i]['days_to_cancel'] >= 1 or enrollments[i]['days_to_cancel'] == None:
        missing_but_remains.add(i)
print missing_but_remains
print(len(missing_but_remains))

set([1378, 467, 468])
3


In [72]:
for i in list(missing_but_remains):
    print(enrollments[i])

{u'status': u'current', u'is_udacity': True, u'is_canceled': False, u'join_date': datetime.datetime(2015, 2, 25, 0, 0), u'account_key': u'1101', u'cancel_date': None, u'days_to_cancel': None}
{u'status': u'canceled', u'is_udacity': True, u'is_canceled': True, u'join_date': datetime.datetime(2015, 1, 10, 0, 0), u'account_key': u'1304', u'cancel_date': datetime.datetime(2015, 3, 10, 0, 0), u'days_to_cancel': 59}
{u'status': u'canceled', u'is_udacity': True, u'is_canceled': True, u'join_date': datetime.datetime(2015, 3, 10, 0, 0), u'account_key': u'1304', u'cancel_date': datetime.datetime(2015, 6, 17, 0, 0), u'days_to_cancel': 99}


## Tracking Down the Remaining Problems

In [73]:
# Create a set of the account keys for all Udacity test accounts
udacity_test_accounts = set()
for enrollment in enrollments:
    if enrollment['is_udacity']:
        udacity_test_accounts.add(enrollment['account_key'])
len(udacity_test_accounts)

6

In [74]:
# Given some data with an account_key field, removes any records corresponding to Udacity test accounts
def remove_udacity_accounts(data):
    non_udacity_data = []
    for data_point in data:
        if data_point['account_key'] not in udacity_test_accounts:
            non_udacity_data.append(data_point)
    return non_udacity_data

In [75]:
# Remove Udacity test accounts from all three tables
non_udacity_enrollments = remove_udacity_accounts(enrollments)
non_udacity_engagement = remove_udacity_accounts(daily_engagement)
non_udacity_submissions = remove_udacity_accounts(project_submissions)

print len(non_udacity_enrollments)
print len(non_udacity_engagement)
print len(non_udacity_submissions)

1622
135656
3634


### See lesson: Number of visits in the first week

In [ ]:
for engagement_record in non_udacity_engagement:
    if engagement_record['num_courses_visited'] > 0:
        engagement_record['has_visited'] = 1
    else:
        engagement_record['has_visited'] = 0

## Refining the Question

In [106]:
#####################################
#                 6                 #
#####################################

## Create a dictionary named paid_students containing all students who either
## haven't canceled yet or who remained enrolled for more than 7 days. The keys
## should be account keys, and the values should be the date the student enrolled.

paid_students = {}
for enrollment in non_udacity_enrollments:
    if enrollment['days_to_cancel'] == None or enrollment['days_to_cancel'] > 7:
        account_key = enrollment['account_key']
        enrollment_date = enrollment['join_date']
        #temp = {'account_key': enrollment['account_key'], 'enrollment_date': enrollment['join_date']}
        #paid_students.append(temp)
        if account_key not in paid_students or enrollment_date > paid_students[account_key]:
            paid_students[account_key] = enrollment_date

In [103]:
print paid_students
#print len(unique_values(paid_students, 'account_key'))
print len(paid_students)

{u'1200': datetime.datetime(2015, 3, 4, 0, 0), u'1175': datetime.datetime(2015, 4, 2, 0, 0), u'1269': datetime.datetime(2015, 8, 21, 0, 0), u'669': datetime.datetime(2015, 5, 12, 0, 0), u'1257': datetime.datetime(2015, 7, 9, 0, 0), u'344': datetime.datetime(2015, 1, 11, 0, 0), u'345': datetime.datetime(2015, 1, 7, 0, 0), u'346': datetime.datetime(2014, 12, 8, 0, 0), u'347': datetime.datetime(2015, 4, 5, 0, 0), u'340': datetime.datetime(2015, 4, 1, 0, 0), u'341': datetime.datetime(2015, 5, 10, 0, 0), u'342': datetime.datetime(2014, 12, 5, 0, 0), u'343': datetime.datetime(2014, 12, 7, 0, 0), u'811': datetime.datetime(2015, 5, 6, 0, 0), u'812': datetime.datetime(2015, 7, 9, 0, 0), u'348': datetime.datetime(2015, 2, 5, 0, 0), u'349': datetime.datetime(2015, 4, 3, 0, 0), u'816': datetime.datetime(2015, 8, 20, 0, 0), u'918': datetime.datetime(2015, 4, 6, 0, 0), u'1149': datetime.datetime(2015, 4, 1, 0, 0), u'719': datetime.datetime(2014, 11, 17, 0, 0), u'718': datetime.datetime(2015, 5, 8, 0

## Getting Data from First Week

In [180]:
# Takes a student's join date and the date of a specific engagement record,
# and returns True if that engagement record happened within one week
# of the student joining.
def within_one_week(join_date, engagement_date):
    time_delta = engagement_date - join_date
    #return time_delta.days < 7
    return time_delta.days < 7 and time_delta.days >= 0

In [181]:
#####################################
#                 7                 #
#####################################

## Create a list of rows from the engagement table including only rows where
## the student is one of the paid students you just found, and the date is within
## one week of the student's join date.

paid_engagement_in_first_week= []
for engagement in daily_engagement:
    key_temp = engagement['account_key']
    if key_temp in set(paid_students):
        if within_one_week(paid_students[key_temp], engagement['utc_date']):
            #temp = {'account_key': key_temp, 'utc_date': engagement['utc_date']}
            paid_engagement_in_first_week.append(engagement)

In [182]:
#print paid_engagement_in_first_week
print paid_engagement_in_first_week[:5]
print len(paid_engagement_in_first_week)

[{u'lessons_completed': 0, u'num_courses_visited': 1, u'total_minutes_visited': 11.6793745, u'projects_completed': 0, 'account_key': u'0', u'utc_date': datetime.datetime(2015, 1, 9, 0, 0)}, {u'lessons_completed': 0, u'num_courses_visited': 2, u'total_minutes_visited': 37.2848873333, u'projects_completed': 0, 'account_key': u'0', u'utc_date': datetime.datetime(2015, 1, 10, 0, 0)}, {u'lessons_completed': 0, u'num_courses_visited': 2, u'total_minutes_visited': 53.6337463333, u'projects_completed': 0, 'account_key': u'0', u'utc_date': datetime.datetime(2015, 1, 11, 0, 0)}, {u'lessons_completed': 0, u'num_courses_visited': 1, u'total_minutes_visited': 33.4892696667, u'projects_completed': 0, 'account_key': u'0', u'utc_date': datetime.datetime(2015, 1, 12, 0, 0)}, {u'lessons_completed': 0, u'num_courses_visited': 1, u'total_minutes_visited': 64.7796776667, u'projects_completed': 0, 'account_key': u'0', u'utc_date': datetime.datetime(2015, 1, 13, 0, 0)}]
6919


## Exploring Student Engagement

In [212]:
from collections import defaultdict

# Create a dictionary of engagement grouped by student.
# The keys are account keys, and the values are lists of engagement records.
engagement_by_account = defaultdict(list)
for engagement_record in paid_engagement_in_first_week:
    account_key = engagement_record['account_key']
    engagement_by_account[account_key].append(engagement_record)
    
## ALTERNATIVE
def group_data(data, key_name):
    grouped_data = defaultdict(list)
    for data_point in data:
        key = data_point[key_name]
        grouped_data[key].append(data_point)
    return grouped_data

engagement_by_account = group_data(paid_engagement_in_first_week, 'account_key')

In [213]:
print engagement_by_account

defaultdict(<type 'list'>, {u'1200': [{u'lessons_completed': 1, u'num_courses_visited': 2, u'total_minutes_visited': 114.853432, u'projects_completed': 0, 'account_key': u'1200', u'utc_date': datetime.datetime(2015, 3, 4, 0, 0)}, {u'lessons_completed': 0, u'num_courses_visited': 1, u'total_minutes_visited': 43.4168625, u'projects_completed': 0, 'account_key': u'1200', u'utc_date': datetime.datetime(2015, 3, 5, 0, 0)}, {u'lessons_completed': 0, u'num_courses_visited': 1, u'total_minutes_visited': 187.776832833, u'projects_completed': 0, 'account_key': u'1200', u'utc_date': datetime.datetime(2015, 3, 6, 0, 0)}, {u'lessons_completed': 0, u'num_courses_visited': 1, u'total_minutes_visited': 150.081577333, u'projects_completed': 0, 'account_key': u'1200', u'utc_date': datetime.datetime(2015, 3, 7, 0, 0)}, {u'lessons_completed': 0, u'num_courses_visited': 1, u'total_minutes_visited': 191.61088, u'projects_completed': 0, 'account_key': u'1200', u'utc_date': datetime.datetime(2015, 3, 8, 0, 0)

In [219]:
# Create a dictionary with the total minutes each student spent in the classroom during the first week.
# The keys are account keys, and the values are numbers (total minutes)
total_minutes_by_account = {}
for account_key, engagement_for_student in engagement_by_account.items():
    total_minutes = 0
    for engagement_record in engagement_for_student:
        total_minutes += engagement_record['total_minutes_visited']
    total_minutes_by_account[account_key] = total_minutes
    
## ALTERNATIVE
def sum_grouped_items(grouped_data, field_name):
    summed_data = {}
    for key, data_points in grouped_data.items():
        total = 0
        for data_point in data_points:
            total += data_point[field_name]
        summed_data[key] = total
    return summed_data

total_minutes_by_account = sum_grouped_items(engagement_by_account, 'total_minutes_visited')

In [221]:
import numpy as np

# Summarize the data about minutes spent in the classroom
total_minutes = total_minutes_by_account.values()
print 'Mean:', np.mean(total_minutes)
print 'Standard deviation:', np.std(total_minutes)
print 'Minimum:', np.min(total_minutes)
print 'Maximum:', np.max(total_minutes)

## ALTERNATIVE
def describe_data(data):
    print 'Mean:', np.mean(data)
    print 'Standard deviation:', np.std(data)
    print 'Minimum:', np.min(data)
    print 'Maximum:', np.max(data)
    
describe_data(total_minutes_by_account.values())

Mean: 306.708326753
Standard deviation: 412.996933409
Minimum: 0.0
Maximum: 3564.7332645
Mean: 306.708326753
Standard deviation: 412.996933409
Minimum: 0.0
Maximum: 3564.7332645


In [187]:
print total_minutes_by_account.values()[:10]

[1482.9020456663002, 129.85920366667, 0.0, 726.26075933267, 614.9778563333, 494.88049616599994, 248.12435916639998, 713.3479524992999, 392.473794834, 35.531418]


## Debugging Data Analysis Code

In [188]:
#####################################
#                 8                 #
#####################################

## Go through a similar process as before to see if there is a problem.
## Locate at least one surprising piece of data, output it, and take a look at it.
anomaly = {}
for account in total_minutes_by_account.keys():
    if account > 10000:
        anomaly[account] = account
print anomaly
print len(anomaly)

{u'619': u'619', u'1200': u'1200', u'1269': u'1269', u'198': u'198', u'1144': u'1144', u'1257': u'1257', u'344': u'344', u'0': u'0', u'346': u'346', u'347': u'347', u'340': u'340', u'341': u'341', u'342': u'342', u'343': u'343', u'811': u'811', u'812': u'812', u'504': u'504', u'348': u'348', u'349': u'349', u'816': u'816', u'1253': u'1253', u'1260': u'1260', u'719': u'719', u'718': u'718', u'715': u'715', u'713': u'713', u'712': u'712', u'710': u'710', u'1265': u'1265', u'606': u'606', u'1264': u'1264', u'916': u'916', u'1060': u'1060', u'1062': u'1062', u'1067': u'1067', u'1066': u'1066', u'913': u'913', u'298': u'298', u'299': u'299', u'296': u'296', u'297': u'297', u'294': u'294', u'295': u'295', u'292': u'292', u'293': u'293', u'290': u'290', u'291': u'291', u'591': u'591', u'590': u'590', u'593': u'593', u'592': u'592', u'595': u'595', u'594': u'594', u'597': u'597', u'596': u'596', u'195': u'195', u'194': u'194', u'197': u'197', u'196': u'196', u'191': u'191', u'190': u'190', u'1

In [189]:
## Find student with maximum minutes
student_with_max_minutes = None
max_minutes = 0
for student, total_minutes in total_minutes_by_account.items():
    if total_minutes > max_minutes:
        max_minutes = total_minutes
        student_with_max_minutes = student

print max_minutes

3564.7332645


In [190]:
for engagement_record in paid_engagement_in_first_week:
    if engagement_record['account_key'] == student_with_max_minutes:
        print engagement_record

{u'lessons_completed': 4, u'num_courses_visited': 4, u'total_minutes_visited': 850.519339666, u'projects_completed': 0, 'account_key': u'163', u'utc_date': datetime.datetime(2015, 7, 9, 0, 0)}
{u'lessons_completed': 6, u'num_courses_visited': 6, u'total_minutes_visited': 872.633923334, u'projects_completed': 0, 'account_key': u'163', u'utc_date': datetime.datetime(2015, 7, 10, 0, 0)}
{u'lessons_completed': 6, u'num_courses_visited': 2, u'total_minutes_visited': 777.018903666, u'projects_completed': 0, 'account_key': u'163', u'utc_date': datetime.datetime(2015, 7, 11, 0, 0)}
{u'lessons_completed': 2, u'num_courses_visited': 1, u'total_minutes_visited': 294.568774, u'projects_completed': 0, 'account_key': u'163', u'utc_date': datetime.datetime(2015, 7, 12, 0, 0)}
{u'lessons_completed': 1, u'num_courses_visited': 3, u'total_minutes_visited': 471.2139785, u'projects_completed': 0, 'account_key': u'163', u'utc_date': datetime.datetime(2015, 7, 13, 0, 0)}
{u'lessons_completed': 1, u'num_cour

## Lessons Completed in First Week

In [222]:
#####################################
#                 9                 #
#####################################

## Adapt the code above to find the mean, standard deviation, minimum, and maximum for
## the number of lessons completed by each student during the first week. Try creating
## one or more functions to re-use the code above.

total_lessons_completed_by_account = {}
for account_key, engagement_for_student in engagement_by_account.items():
    lessons_completed = 0
    for engagement_record in engagement_for_student:
        lessons_completed += engagement_record['lessons_completed']
    total_lessons_completed_by_account[account_key] = lessons_completed

## ALTERNATIVE
total_lessons_completed_by_account = sum_grouped_items(engagement_by_account, 'lessons_completed')
    
total_lessons_completed = total_lessons_completed_by_account.values()
print 'Mean:', np.mean(total_lessons_completed)
print 'Standard deviation:', np.std(total_lessons_completed)
print 'Minimum:', np.min(total_lessons_completed)
print 'Maximum:', np.max(total_lessons_completed)

## ALTERNATIVE
describe_data(total_lessons_completed_by_account.values())

    
        
    


Mean: 1.63618090452
Standard deviation: 3.00256129983
Minimum: 0
Maximum: 36
Mean: 1.63618090452
Standard deviation: 3.00256129983
Minimum: 0
Maximum: 36


## Number of Visits in First Week

In [225]:
######################################
#                 10                 #
######################################

## Find the mean, standard deviation, minimum, and maximum for the number of
## days each student visits the classroom during the first week.

## OPTION 1
def sum_grouped_items_v2(grouped_data, field_name):
    summed_data = {}
    for key, data_points in grouped_data.items():
        total = 0
        for data_point in data_points:
            if data_point[field_name] >= 1:
                total += 1
        summed_data[key] = total
    return summed_data

total_num_courses_visited = sum_grouped_items_v2(engagement_by_account, 'num_courses_visited')
print total_num_courses_visited

{u'619': 5, u'1140': 3, u'669': 0, u'428': 5, u'344': 7, u'0': 7, u'346': 3, u'347': 5, u'340': 3, u'341': 2, u'342': 0, u'343': 2, u'811': 5, u'812': 6, u'348': 0, u'349': 2, u'816': 0, u'918': 2, u'1149': 7, u'719': 1, u'718': 0, u'715': 6, u'426': 0, u'713': 2, u'712': 5, u'710': 1, u'661': 2, u'129': 1, u'660': 5, u'422': 5, u'1060': 7, u'1062': 5, u'1067': 1, u'1066': 0, u'671': 3, u'427': 3, u'298': 3, u'299': 0, u'296': 3, u'297': 3, u'294': 4, u'295': 5, u'292': 3, u'293': 5, u'290': 5, u'291': 0, u'591': 0, u'590': 0, u'593': 0, u'592': 6, u'595': 3, u'594': 0, u'597': 6, u'596': 6, u'195': 6, u'194': 6, u'197': 7, u'196': 6, u'191': 4, u'190': 1, u'193': 4, u'192': 5, u'270': 6, u'271': 6, u'272': 7, u'273': 3, u'274': 7, u'275': 2, u'276': 5, u'277': 7, u'278': 0, u'279': 1, u'497': 4, u'524': 0, u'525': 0, u'526': 5, u'527': 3, u'520': 2, u'1019': 4, u'522': 5, u'523': 2, u'1014': 6, u'1015': 2, u'528': 5, u'529': 0, u'1013': 1, u'1235': 3, u'449': 1, u'391': 0, u'1231': 4,

In [ ]:
describe_data(total_num_courses_visited.values())

In [230]:
## OPTION 2
total_num_courses_visited

for account_key, engagement_for_student in engagement_by_account.items():
    for engagement_record in engagement_for_student:
        engagement_record['has_visited'] = 0
        if engagement_record['num_courses_visited'] >= 1:
            engagement_record['has_visited'] = 1

total_num_courses_visited = sum_grouped_items(engagement_by_account, 'has_visited')
print total_num_courses_visited

{u'619': 5, u'1140': 3, u'669': 0, u'428': 5, u'344': 7, u'0': 7, u'346': 3, u'347': 5, u'340': 3, u'341': 2, u'342': 0, u'343': 2, u'811': 5, u'812': 6, u'348': 0, u'349': 2, u'816': 0, u'918': 2, u'1149': 7, u'719': 1, u'718': 0, u'715': 6, u'426': 0, u'713': 2, u'712': 5, u'710': 1, u'661': 2, u'129': 1, u'660': 5, u'422': 5, u'1060': 7, u'1062': 5, u'1067': 1, u'1066': 0, u'671': 3, u'427': 3, u'298': 3, u'299': 0, u'296': 3, u'297': 3, u'294': 4, u'295': 5, u'292': 3, u'293': 5, u'290': 5, u'291': 0, u'591': 0, u'590': 0, u'593': 0, u'592': 6, u'595': 3, u'594': 0, u'597': 6, u'596': 6, u'195': 6, u'194': 6, u'197': 7, u'196': 6, u'191': 4, u'190': 1, u'193': 4, u'192': 5, u'270': 6, u'271': 6, u'272': 7, u'273': 3, u'274': 7, u'275': 2, u'276': 5, u'277': 7, u'278': 0, u'279': 1, u'497': 4, u'524': 0, u'525': 0, u'526': 5, u'527': 3, u'520': 2, u'1019': 4, u'522': 5, u'523': 2, u'1014': 6, u'1015': 2, u'528': 5, u'529': 0, u'1013': 1, u'1235': 3, u'449': 1, u'391': 0, u'1231': 4,

In [231]:
describe_data(total_num_courses_visited.values())

Mean: 2.86733668342
Standard deviation: 2.25519800292
Minimum: 0
Maximum: 7


## Splitting out Passing Students

In [243]:
######################################
#                 11                 #
######################################

## Create two lists of engagement data for paid students in the first week.
## The first list should contain data for students who eventually pass the
## subway project, and the second list should contain data for students
## who do not.

subway_project_lesson_keys = ['746169184', '3176718735']

submissions_by_account = group_data(non_udacity_submissions, 'account_key')

passing_engagement = []
for key in engagement_by_account:
    if key in set(submissions_by_account):
        if submission['lesson_key'] in subway_project_lesson_keys:
            if submission['assigned_rating'] == "PASSED":
                passing_engagement.append(submission)
            
non_passing_engagement = []
for key in engagement_by_account:
    if key in set(submissions_by_account):
        if submission['lesson_key'] in subway_project_lesson_keys:
            if submission['assigned_rating'] != "PASSED":
                non_passing_engagement.append(submission)

print len(passing_engagement)
print len(non_passing_engagement)

0
0


In [232]:
non_udacity_submissions[0]

{u'account_key': u'256',
 u'assigned_rating': u'UNGRADED',
 u'completion_date': datetime.datetime(2015, 1, 16, 0, 0),
 u'creation_date': datetime.datetime(2015, 1, 14, 0, 0),
 u'lesson_key': u'3176718735',
 u'processing_state': u'EVALUATED'}

In [234]:
submissions_by_account

defaultdict(list,
            {u'0': [{u'account_key': u'0',
               u'assigned_rating': u'INCOMPLETE',
               u'completion_date': datetime.datetime(2015, 4, 3, 0, 0),
               u'creation_date': datetime.datetime(2015, 4, 3, 0, 0),
               u'lesson_key': u'3176718735',
               u'processing_state': u'EVALUATED'},
              {u'account_key': u'0',
               u'assigned_rating': u'INCOMPLETE',
               u'completion_date': datetime.datetime(2015, 5, 5, 0, 0),
               u'creation_date': datetime.datetime(2015, 5, 5, 0, 0),
               u'lesson_key': u'3168208620',
               u'processing_state': u'EVALUATED'},
              {u'account_key': u'0',
               u'assigned_rating': u'PASSED',
               u'completion_date': datetime.datetime(2015, 7, 27, 0, 0),
               u'creation_date': datetime.datetime(2015, 7, 27, 0, 0),
               u'lesson_key': u'3168208620',
               u'processing_state': u'EVALUATED'},
   

## Comparing the Two Student Groups

In [ ]:
######################################
#                 12                 #
######################################

## Compute some metrics you're interested in and see how they differ for
## students who pass the subway project vs. students who don't. A good
## starting point would be the metrics we looked at earlier (minutes spent
## in the classroom, lessons completed, and days visited).

## Making Histograms

In [ ]:
######################################
#                 13                 #
######################################

## Make histograms of the three metrics we looked at earlier for both
## students who passed the subway project and students who didn't. You
## might also want to make histograms of any other metrics you examined.

## Improving Plots and Sharing Findings

In [ ]:
######################################
#                 14                 #
######################################

## Make a more polished version of at least one of your visualizations
## from earlier. Try importing the seaborn library to make the visualization
## look better, adding axis labels and a title, and changing one or more
## arguments to the hist() function.